In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import torch
from heat_solver import HeatSolver
from inverse_solver import InverseSolver
from utils import create_conductivity_field, _boundary_mask, sine_source
from utils import SimpleSigma, SigmoidSigma

from run import get_boundary_conditions

# Test

In [77]:
# === Params ===
M = 10
T = 1.0
device = 'cpu'
max_sigma = 5
alpha = 0.01
sigma_0 = 1.0  # Initial guess
lr = 1e-1
omega = 2.0 * 3.1415   # Frequency for sine source
noise_level = 0.01
max_iters = 5000
tol = 1e-3
pattern = 'linear'


source_func = lambda x, y, t: sine_source(x, y, t, omega)
sigma_gt = create_conductivity_field(M, pattern=pattern, device=device)

In [78]:
# Generate boundary observations with noise
u_b_gt = get_boundary_conditions(sigma_gt, source_func, T, max_sigma, device=device)
u_b = (1 + noise_level * torch.randn_like(u_b_gt)) * u_b_gt

In [80]:
sigma = SimpleSigma(M, sigma_0)

In [81]:
# Inverse solver
inverse_solver = InverseSolver(
    sigma,
    u_b_gt=u_b,
    source_func=source_func,
    M=M,
    T=T,
    n_steps=u_b.shape[0] - 1,
    alpha=alpha,
    sigma_0=sigma_0,
    device=device
)

In [82]:
final_sigma, total_loss_history, boundary_loss_history, regularization_loss_history = inverse_solver.solve(
    max_iters=max_iters,
    tol=tol,
    print_info=True
)

  0%|          | 0/5000 [00:00<?, ?it/s]

1.0 4000
Grid: 10x10, Time step: 0.000250, Steps: 4000


  0%|          | 0/5000 [00:04<?, ?it/s]

Converged at iteration 0, loss: 0.000101
